# X-matching

Here we will test the cross-matching of catalogs using the very same catalog.
Clearly, the matching with itself will --should-- match the very same object at each position; for this reason --because of the known, trivial result-- we should enjoy the moment to handle pre-/post-processing tasks.

For instance, using the [booq] library we should:
* plot the data points (i.e, objects) position
* cutout a region of the sky to sample the dataset
* shift objects position
* run the x-match pipeline using different methods
 * nearest-neighbor
 * great-circle
 * "la massa"
   * plot distances between the matching pairs

[booq]: https://github.com/chbrandt/booq

In [1]:
import booq

## The Data

We are going to use the `cs82` catalog.

In [2]:
!ls cs82

changelog.txt	  contents.cfg				   metadata.csv
columns_all	  CS82 data inspection (incomplete).ipynb  samples
columns_morphoto  cs82_photoauto.fits
columns_world	  gavo


In [3]:
filedata='cs82/cs82_photoauto.fits'
filemeta='cs82/metadata.csv'

### As ATable

In [4]:
from booq.table import ATable
cs82_test = ATable.read(data=filedata, metadata=filemeta, rows=1E4)

In [5]:
cs82_test.metatable

,description,unit,ucd,dtype,nil
colname,,,,,
OBJID,ObjectID ($index),--,"((meta, id), (meta, main))",int64,--
ALPHA_J2000,Right Ascencion,deg,"((pos, eq, ra), (meta, main))",float64,--
DELTA_J2000,Declination,deg,"((pos, eq, dec), (meta, main))",float64,--
MAG_AUTO,Magnitude filter i,mag,"((phot, mag), (em, opt, I))",float32,--
MAGERR_AUTO,Magnitude error filter i,mag,"((stat, error), (phot, mag), (em, opt, I))",float32,--
CLASS_STAR,--,--,"((src, class, starGalaxy))",float32,--
FLUX_RADIUS,--,px,"((phot, flux), (em, opt, I))",float32,--
FLAGS,Quality of photometry,--,"((meta, code, qual))",int16,--


In [6]:
cs82_test

OBJID,ALPHA_J2000,DELTA_J2000,MAG_AUTO,MAGERR_AUTO,CLASS_STAR,FLUX_RADIUS,FLAGS
,deg,deg,mag,mag,,px,
int64,float64,float64,float32,float32,float32,float32,int16
2609,-0.774542651339,-0.971344230377,23.6655,0.136823,0.79214,2.79215,0
5032,-1.40109155171,-0.940996135742,24.1153,0.133869,0.108331,1.60765,0
6245,-0.952408197038,-0.924954357991,23.8837,0.120466,0.535358,3.69384,2
6704,-1.39575648939,-0.918731640744,22.282,0.0375658,0.0188971,2.71909,0
10481,-1.30100007027,-0.870724049331,22.9581,0.0944459,0.000140369,4.52054,0
12709,-1.3504286251,-0.847851230977,22.9601,0.0606773,0.122526,2.06206,0
17905,-0.542615991536,-0.794216579,22.5412,0.0493326,0.000394834,3.26503,0
21314,-0.684256347819,-0.754340863016,24.7529,0.229552,0.396414,2.03572,0


### As ADataFrame

In [7]:
df = cs82_test.to_dataframe()

In [8]:
#df.metatable

In [9]:
df

,OBJID,ALPHA_J2000,DELTA_J2000,MAG_AUTO,MAGERR_AUTO,CLASS_STAR,FLUX_RADIUS,FLAGS
0,2609,-0.774543,-0.971344,23.665508,0.136823,0.792140,2.792147,0
1,5032,-1.401092,-0.940996,24.115332,0.133869,0.108331,1.607653,0
2,6245,-0.952408,-0.924954,23.883665,0.120466,0.535358,3.693837,2
3,6704,-1.395756,-0.918732,22.281969,0.037566,0.018897,2.719090,0
4,10481,-1.301000,-0.870724,22.958069,0.094446,0.000140,4.520538,0
5,12709,-1.350429,-0.847851,22.960114,0.060677,0.122526,2.062063,0
6,17905,-0.542616,-0.794217,22.541241,0.049333,0.000395,3.265028,0
7,21314,-0.684256,-0.754341,24.752857,0.229552,0.396414,2.035718,0
8,22954,-0.906235,-0.734064,22.574326,0.073038,0.031559,2.661597,0
9,26035,-0.823958,-0.698893,20.575359,0.013719,0.028837,4.161399,2


## Plot objects position

In [11]:
from booq import plot
plot.set_device('notebook')

/home/chbrandt/fido/repos/booq/booq/plot/mybokeh/__init__.py:433: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(False, "Not implemented yet")
/home/chbrandt/.conda/envs/booq/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


In [12]:
from booq.plot import PlotScatter
p = PlotScatter()
ds_label = p.add_dataset(df,label='catalog sample',x='ALPHA_J2000',y='DELTA_J2000')
p.plot(ds_label)
p.show()

Loading BokehJS ...

## Select a sub-sample

We'll select now a sample based on their position, the goal is to have data points close to each other, to eventually x-match such positions.

In [13]:
ra = (0,5)
dec= (0,1)
_ir = df.ALPHA_J2000.between(*ra)
_id = df.DELTA_J2000.between(*dec)
_sel = _ir & _id
df_sample = df.loc[_sel]

In [14]:
ds_label = p.add_dataset(df_sample,label='little_square',x='ALPHA_J2000',y='DELTA_J2000')
p.plot(ds_label)
p.show()

## Shift sub-sample position

Eventually, we'll x-match the objects of "sample" with their neighbors; because we want to assess the typical distance within *CS82* objects we will crossmatch the same catalog on each side.
Clearly we cannot steady match those objects, otherwise objects will all match themselves, at distance zero; we need to shift them from a certain amount.

What we'll to do now is to shift all the objects in "sample" to another region of the plot where the density is uniform.
This way

In [15]:
ra_s = (-35,-30)
dec_s= (-1,0)
#region = zip(ra,dec)
xc = (ra_s[1]-ra_s[0])/2 + ra_s[0]
yc = (dec_s[1]-dec_s[0])/2 + dec_s[0]
xl = ra_s[1]-ra_s[0]
yl = dec_s[1]-dec_s[0]
region = dict(center=(xc,yc),sides=(xl,yl))
reg_label = p.add_region(region,label='counter region')
p.plot(reg_label)
p.show()

In [16]:
# EXTRACTED BOKEH-SELECTED POINTS FROM PLOT

In [17]:
ra_center_sample = ra[0] + (ra[1] - ra[0])/2
dec_center_sample = dec[0] + (dec[1] - dec[0])/2
ra_shift = xc - ra_center_sample
dec_shift = yc - dec_center_sample
print('Shift position: RA:{} ; Dec:{}'.format(ra_shift,dec_shift))

Shift position: RA:-35.0 ; Dec:-1.0


In [18]:
df_shift = df_sample.copy()
df_shift.ALPHA_J2000 = df_sample.ALPHA_J2000 + ra_shift
df_shift.DELTA_J2000 = df_sample.DELTA_J2000 + dec_shift
df_shift.describe()

              OBJID  ALPHA_J2000  DELTA_J2000    MAG_AUTO  MAGERR_AUTO  \
count  2.560000e+02   256.000000   256.000000  256.000000   256.000000   
mean   1.223465e+07   -32.620157    -0.539295   23.470467     0.112069   
std    2.882092e+06     1.411306     0.264736    1.457863     0.070940   
min    7.409026e+06   -34.977540    -0.999489   18.151360     0.001116   
25%    9.672469e+06   -33.831197    -0.755851   22.863778     0.050357   
50%    1.187874e+07   -32.579358    -0.541875   23.780987     0.110617   
75%    1.535903e+07   -31.459735    -0.315408   24.484078     0.162663   
max    1.558138e+07   -30.041352    -0.065046   25.805803     0.312472   

       CLASS_STAR  FLUX_RADIUS       FLAGS  
count  256.000000   256.000000  256.000000  
mean     0.258787     2.900509    0.347656  
std      0.292045     1.304429    0.907612  
min      0.000126     1.253912    0.000000  
25%      0.020178     2.164235    0.000000  
50%      0.086092     2.607055    0.000000  
75%      0.497105 

In [19]:
_ir = df.ALPHA_J2000.between(*ra_s)
_id = df.DELTA_J2000.between(*dec_s)
_sel = _ir & _id
df_counter = df.loc[_sel]
df_counter.describe()

              OBJID  ALPHA_J2000  DELTA_J2000    MAG_AUTO  MAGERR_AUTO  \
count  3.470000e+02   347.000000   347.000000  347.000000   347.000000   
mean   3.890520e+06   -32.539351    -0.511057   23.411972     0.105416   
std    4.085083e+05     1.405469     0.278071    1.622708     0.075399   
min    3.140234e+06   -34.971781    -0.995969   17.783897     0.000652   
25%    3.527720e+06   -33.701883    -0.749582   22.622023     0.038822   
50%    3.931886e+06   -32.720679    -0.519132   23.800619     0.096666   
75%    4.320219e+06   -31.372246    -0.279213   24.672006     0.163996   
max    4.615349e+06   -30.011974    -0.000758   25.935667     0.307629   

       CLASS_STAR  FLUX_RADIUS       FLAGS  
count  347.000000   347.000000  347.000000  
mean     0.341926     2.733551    0.345821  
std      0.345323     1.316458    0.877631  
min      0.000133     0.990985    0.000000  
25%      0.022910     1.904746    0.000000  
50%      0.195239     2.417965    0.000000  
75%      0.635009 

In [20]:
ds_label = p.add_dataset(df_counter,label='counter_square',x='ALPHA_J2000',y='DELTA_J2000')
p.plot(ds_label)
p.show()

In [21]:
ds_label = p.add_dataset(df_shift,label='little_square-shifted',x='ALPHA_J2000',y='DELTA_J2000')
p.plot(ds_label)
p.show()

## X-Match

Now the time to actually cross-match the samples has arrived.

In [22]:
from booq.pipelines import xmatch

### Nearest Neighbour

We should apply the nearest-neighbor first hand and see how the matching distances plot.
I suspect that the distances-distribution will show a peak somewhere between the objects mean position error and the typical value --if there is-- to objects separation.

In [23]:
columns = dict(ra='ALPHA_J2000', dec='DELTA_J2000', id='OBJID')
df_match = xmatch.xmatch(df_shift, df_counter,
                         columns_A=columns, columns_B=columns,
                         method='nn')
df_match.describe()

/home/chbrandt/.conda/envs/booq/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/chbrandt/.conda/envs/booq/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


                  A                                                  \
              OBJID ALPHA_J2000 DELTA_J2000    MAG_AUTO MAGERR_AUTO   
count  2.560000e+02  256.000000  256.000000  256.000000  256.000000   
mean   1.223465e+07  -32.620157   -0.539295   23.470467    0.112069   
std    2.882092e+06    1.411306    0.264736    1.457863    0.070940   
min    7.409026e+06  -34.977540   -0.999489   18.151360    0.001116   
25%    9.672469e+06  -33.831197   -0.755851   22.863778    0.050357   
50%    1.187874e+07  -32.579358   -0.541875   23.780987    0.110617   
75%    1.535903e+07  -31.459735   -0.315408   24.484078    0.162663   
max    1.558138e+07  -30.041352   -0.065046   25.805803    0.312472   

                                                      B              \
       CLASS_STAR FLUX_RADIUS       FLAGS         OBJID ALPHA_J2000   
count  256.000000  256.000000  256.000000  1.340000e+02  134.000000   
mean     0.258787    2.900509    0.347656  3.934047e+06  -32.649369   
std  

In [24]:
from bokeh.io import show

from booq.plot import PlotHisto
p = PlotHisto()
ds_label = p.add_dataset(df_match,label='distances',x=('AB','dist'))
p.plot(ds_label)
p.show()


Loading BokehJS ...

### Great Circle

In [25]:
from astropy import units
r = 0.1 * units.degree

columns = dict(ra='ALPHA_J2000', dec='DELTA_J2000', id='OBJID')
df_match = xmatch.xmatch(df_shift, df_counter,
                         columns_A=columns, columns_B=columns,
                         method='gc', radius=r)


/home/chbrandt/.conda/envs/booq/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/chbrandt/.conda/envs/booq/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [26]:
df_match.describe()

                   A                                                  \
               OBJID ALPHA_J2000 DELTA_J2000    MAG_AUTO MAGERR_AUTO   
count   2.560000e+02  256.000000  256.000000  256.000000  256.000000   
unique           NaN         NaN         NaN         NaN         NaN   
top              NaN         NaN         NaN         NaN         NaN   
freq             NaN         NaN         NaN         NaN         NaN   
mean    1.223465e+07  -32.620157   -0.539295   23.470467    0.112069   
std     2.882092e+06    1.411306    0.264736    1.457863    0.070940   
min     7.409026e+06  -34.977540   -0.999489   18.151360    0.001116   
25%     9.672469e+06  -33.831197   -0.755851   22.863778    0.050357   
50%     1.187874e+07  -32.579358   -0.541875   23.780987    0.110617   
75%     1.535903e+07  -31.459735   -0.315408   24.484078    0.162663   
max     1.558138e+07  -30.041352   -0.065046   25.805803    0.312472   

                                                       B       

In [27]:
print(df_match)

               A                                                            \
           OBJID ALPHA_J2000 DELTA_J2000   MAG_AUTO MAGERR_AUTO CLASS_STAR   
1            NaN         NaN         NaN        NaN         NaN        NaN   
2            NaN         NaN         NaN        NaN         NaN        NaN   
3            NaN         NaN         NaN        NaN         NaN        NaN   
5            NaN         NaN         NaN        NaN         NaN        NaN   
6            NaN         NaN         NaN        NaN         NaN        NaN   
8            NaN         NaN         NaN        NaN         NaN        NaN   
9            NaN         NaN         NaN        NaN         NaN        NaN   
10           NaN         NaN         NaN        NaN         NaN        NaN   
11           NaN         NaN         NaN        NaN         NaN        NaN   
12           NaN         NaN         NaN        NaN         NaN        NaN   
13           NaN         NaN         NaN        NaN         NaN 

In [28]:
from bokeh.io import show

from booq.plot import PlotHisto
p = PlotHisto()
ds_label = p.add_dataset(df_match,label='distances',x=('AB','dist'))
p.plot(ds_label)
p.show()


Loading BokehJS ...

## KNN

Out of curiosity, let us compute the typical, or *mean* distance between the first `k` neighbours of each object in the catalog; the entire catalog.


In fact, I want to know the average distance of the first neighbour; although, `k=2` will be used because we are going to match the catalog with itself.

In [29]:
from sklearn.neighbors import NearestNeighbors

X = df[['ALPHA_J2000','DELTA_J2000']].values
nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

In [30]:
dists = distances.transpose()

In [31]:
from bokeh.io import show

from booq.plot import PlotHisto
p = PlotHisto()
ds_label = p.add_dataset(dists,label='1st-neighbour distances',x=1)
p.plot(ds_label)
ds_label = p.add_dataset(dists,label='2nd-neighbour distances',x=2)
p.plot(ds_label)
p.show()


Loading BokehJS ...

#TODO:
* plot legend: WRONG and missing labels; labels should be the name/label given to `add_dataset`
* plot axes: x-axis label is missing; axis label should be either the `x` argument --if--, or the default: "bins"
